In [1]:
# 2 weeks together
import pandas as pd
import numpy as np
import datetime
import os
# Add the new folders of Jian later

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
last_saturday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday()+2) #+7

write_folder="/home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_"+str(last_saturday)+"/"
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)

week_beginning=last_saturday-datetime.timedelta(days=14) 

recent_2_weeks=[last_saturday-datetime.timedelta(days=x*7) for x in range(2)]
recent_2_weeks

[datetime.date(2020, 2, 8), datetime.date(2020, 2, 1)]

In [3]:
# Checking for the recent weeks that haven't moved to the folder
possible_recent_folders=["/home/jian/BigLots/MediaStorm_"+str(x)+"/" for x in recent_2_weeks]
recent_file_list=[]
for dirc in possible_recent_folders:
    list_recent=[x for x in list(recursive_file_gen(dirc)) if ("Master" in x) & (".txt" in x) ]
    recent_file_list=recent_file_list+list_recent
recent_file_df=pd.DataFrame({"path":recent_file_list,"date":[datetime.datetime.strptime(x.split("MasterWeekly")[1][:8],"%Y%m%d").date()-datetime.timedelta(days=3) for x in recent_file_list]},index=[x for x in range(len(recent_file_list))])


list_1_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/")) if ("Master" in x) & (".txt" in x) ]
list_2_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2020_by_weeks/")) if ("Master" in x) & (".txt" in x) ]
list_1_after_201806=list_1_after_201806+list_2_after_201806
folder_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-2].split("_")[1],"%Y-%m-%d").date() for x in list_1_after_201806]
df_1_after_201806=pd.DataFrame({"date":folder_date,"path":list_1_after_201806},index=[x for x in range(len(list_1_after_201806))])
df_1_after_201806['date'].apply(lambda x: x.weekday()).unique()
df_1_after_201806=df_1_after_201806.sort_values("date").reset_index()
df_1_after_201806=df_1_after_201806[df_1_after_201806['date']>week_beginning]
del df_1_after_201806['index']

new_2_biweekly_files=df_1_after_201806.append(recent_file_df)

print(len(new_2_biweekly_files))
new_2_biweekly_files=new_2_biweekly_files['path'].unique().tolist()

2


/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [4]:
new_2_biweekly_files

['/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-02-01/MediaStormMasterWeekly20200204-111949-422.txt',
 '/home/jian/BigLots/MediaStorm_2020-02-08/MediaStormMasterWeekly20200211-121122-184.txt']

In [5]:
if len(new_2_biweekly_files)==2: # Changed to weekly
    df_all_new_master = pd.DataFrame()
    for file in new_2_biweekly_files:
        df = pd.read_csv(file,nrows = None,sep= '|',usecols = ['customer_id_hashed','email_address_hash','customer_zip_code'],
                         dtype =str)
        df_all_new_master = df_all_new_master.append(df,ignore_index = True)
        print(df_all_new_master.shape)
    print(len(df_all_new_master.index))
    df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
    print(len(df_all_new_master.index))


(88226, 3)
(172107, 3)
172107
172107


In [7]:
# Remove previous week ids

previous_id_files_lists=list(recursive_file_gen("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//"))

# previous_id_files_lists=[x for x in previous_id_files_lists if "Copy of 48 Stores_727.csv" not in x]
previous_id_files_lists=[x for x in previous_id_files_lists if ".csv" in x]
print(len(previous_id_files_lists))

273


In [8]:
previous_id_files_lists

['/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Upda

In [9]:
import glob
uploaded_new_seg=list(recursive_file_gen("/home/jian/celery/BiWeekly_New_SignUps/monthly_update/"))
uploaded_new_seg=[x for x in uploaded_new_seg if ".csv" in x]
print("len(uploaded_new_seg)"+str(len(uploaded_new_seg)))
previous_id_files_lists=previous_id_files_lists+uploaded_new_seg

len(uploaded_new_seg)72


In [10]:
# Exclude the quarterly regular segments


In [11]:
for file_uploaded in previous_id_files_lists:
    df = pd.read_csv(file_uploaded,usecols = ['email_address_hash'])
    previous_email_set=set(df['email_address_hash'].tolist())
    df_all_new_master = df_all_new_master[~df_all_new_master['email_address_hash'].isin(previous_email_set)]
    print(datetime.datetime.now(),len(df_all_new_master.index),file_uploaded)

2020-02-12 10:46:24.556477 172107 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-02-12 10:46:25.485711 172107 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-02-12 10:46:25.865400 172107 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-02-12 10:46:26.037574 172107 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-02-12 10:46:34.268762 172103 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded

2020-02-12 10:47:16.837219 172093 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_S_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-02-12 10:47:17.044970 172093 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_S_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-02-12 10:47:17.120230 172093 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_L_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-02-12 10:47:17.196217 172093 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_L_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-02-12 10:47:17.268238 172093 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Activ

2020-02-12 10:47:45.443781 172091 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_L_S_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-02-12 10:47:45.554198 172091 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_L_S_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-02-12 10:47:45.673258 172091 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_M_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-02-12 10:47:45.801600 172091 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_M_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-02-12 10:47:45.912962 172091 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Activ

2020-02-12 10:48:00.507385 172084 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_M_S_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-02-12 10:48:00.675925 172084 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_M_S_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-02-12 10:48:03.008744 172084 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//C_all_quarterly_segments_RFM20Q1.csv
2020-02-12 10:48:03.178349 172084 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Lapsed1318_L_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-02-12 10:48:03.291559 172084 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Lapsed1318_L_P_Above50K_MultiFamilyOrCondo_Competitors

2020-02-12 10:48:16.329100 172082 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-02-12 10:48:21.949348 172081 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_H_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-02-12 10:48:22.982849 172079 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_H_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-02-12 10:48:25.110795 172078 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_H_P_Above50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-02-12 10:48:25.383642 172078 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_H_

2020-02-12 10:48:39.666369 172076 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_L_P_Below50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-02-12 10:48:39.855880 172076 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_L_P_Below50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-02-12 10:48:39.937918 172076 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_L_P_Below50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-02-12 10:48:40.028823 172076 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_L_P_Below50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-02-12 10:48:40.339221 172076 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_L

2020-02-12 10:48:56.029442 172074 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_M_S_Above50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-02-12 10:48:56.162135 172074 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_M_S_Above50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-02-12 10:48:57.054384 172074 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_M_S_Above50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-02-12 10:48:57.175037 172074 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_M_S_Above50K_SingleFamily_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-02-12 10:48:57.420322 172074 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_M_S_Unkn

2020-02-12 10:49:14.856549 172072 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-02-23/NewReward_2019-02-23_P.csv
2020-02-12 10:49:15.094329 172072 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-02-23/NewReward_2019-02-23_S.csv
2020-02-12 10:49:15.232664 172072 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-02-23/NewReward_2019-02-23_T.csv
2020-02-12 10:49:17.531336 172072 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-03-23/NewReward_2019-03-23_P.csv
2020-02-12 10:49:17.985372 172072 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-03-23/NewReward_2019-03-23_S.csv
2020-02-12 10:49:18.099398 172071 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-03-23/NewReward_2019-03-23_T.csv
2020-02-12 10:49:19.206046 172071 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-04-06/NewReward_2019-04-06_P.csv
2020-02-12 10:49:19.344089 172071 /home/jian/celery/BiWeekly_New_Sign

2020-02-12 10:50:01.513610 172068 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-01-11/NewReward_2020-01-11_T.csv
2020-02-12 10:50:02.059756 172068 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-01-25/NewReward_2020-01-25_P.csv
2020-02-12 10:50:02.220274 172068 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-01-25/NewReward_2020-01-25_S.csv
2020-02-12 10:50:02.630047 172068 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-01-25/NewReward_2020-01-25_T.csv


In [12]:
df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
df_all_new_master = df_all_new_master.drop_duplicates('customer_id_hashed')
print(len(df_all_new_master.index))

172067


In [13]:
zipmap = pd.ExcelFile('/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx')
zipmap=zipmap.parse("unique_zips_full_footprint",dtype=str)
zipmap['zip_type']=zipmap['zip_type'].replace("zips_10","trans_S")

zipmap['zipcodegroup'] = zipmap['zip_type'].apply(lambda x: x.split("_")[1])
print(zipmap.shape)
print(zipmap['zip_cd'].nunique())

zipmap=zipmap.rename(columns={"zip_cd":"customer_zip_code"})

zipmap=zipmap[['customer_zip_code','zipcodegroup']]
zipmap.head(2)

(12980, 3)
12980


,customer_zip_code,zipcodegroup
0,75040,P
1,20743,P


In [14]:
df_all_new_master = pd.merge(df_all_new_master,zipmap,on ='customer_zip_code',how = 'left' )
print(df_all_new_master['zipcodegroup'].unique())

df_all_new_master['zipcodegroup'].fillna('T',inplace = True)
print(df_all_new_master['zipcodegroup'].unique())

last_saturday_str="_"+str(last_saturday).split("-")[1]+str(last_saturday).split("-")[2]+str(last_saturday).split("-")[0][2:]+"_"

for revenue_flag in ['P','S','T']:
    df = df_all_new_master[df_all_new_master['zipcodegroup']==revenue_flag]
    df = df[['customer_id_hashed','email_address_hash','customer_zip_code']]
    df['segment_new_signups'] = 'NewReward' + last_saturday_str + revenue_flag
    print(revenue_flag,len(df.index))
    df.to_csv(write_folder+'NewReward_'+str(last_saturday)+'_'+revenue_flag+'.csv',index = False)
# To be uploaded to lr-big-lots /uploads/big_lots_onboarding

['S' nan 'P']
['S' 'T' 'P']
P 74696
S 30549
T 66822


In [15]:
df_all_new_master.groupby(['zipcodegroup'])['email_address_hash'].count().to_frame().reset_index()

,zipcodegroup,email_address_hash
0,P,74696
1,S,30549
2,T,66822


In [16]:
df.head(2)

,customer_id_hashed,email_address_hash,customer_zip_code,segment_new_signups
1,da6a56db4414fe9fd5847c03ff3cc0909cdc7e3ca21670...,c9f9bbfa68d9a2c2124939957eb48a6d8f5445cee9f203...,22032,NewReward_020820_T
2,58a0ca49cf61d36b1e0c95eebf84adfb6500acbddd5fb9...,4a019858766b0fb508959c2044fbf0e6187c38b828d050...,NaN,NewReward_020820_T


In [17]:
print(datetime.datetime.now())

2020-02-12 10:50:07.854662


In [18]:
print(datetime.datetime.now())

2020-02-12 10:50:07.860864


In [19]:
import paramiko
import glob

host = "files.liveramp.com"
port = 22
password = "Biglots2018!"
username = "lr-big-lots"

transport = paramiko.Transport((host, port))

transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

# local_path defined above before saving the local txt
local_list=glob.glob(write_folder+"*.csv")
local_list=[x for x in local_list if 'NewReward_'+str(last_saturday)+'_' in x]

for local_path in local_list:
    remote_path="/uploads/big_lots_onboarding/"+os.path.basename(local_path)
    sftp.put(local_path,remote_path)
sftp.close()
transport.close()
print("Done: ",datetime.datetime.now())

Done:  2020-02-12 10:51:16.858461
